<a href="https://colab.research.google.com/github/jeevisha3008/Text-Mining/blob/master/topic_model_web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scraping-BBC-for-all-articles" data-toc-modified-id="Scraping-BBC-for-all-articles-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Scraping BBC for all articles</a></span></li><li><span><a href="#Data-cleaning-removing-punctuations,quotations,spaces" data-toc-modified-id="Data-cleaning-removing-punctuations,quotations,spaces-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data cleaning-removing punctuations,quotations,spaces</a></span></li><li><span><a href="#Running-the-model" data-toc-modified-id="Running-the-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Running the model</a></span></li><li><span><a href="#Predicting-topic-through-a-Web-App" data-toc-modified-id="Predicting-topic-through-a-Web-App-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Predicting topic through a Web App</a></span><ul class="toc-item"><li><span><a href="#function" data-toc-modified-id="function-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>function</a></span></li></ul></li></ul></div>

In [3]:
!pip install ipywidgets
!pip install python-docx
!pip install beautifulsoup4
!pip install requests
!pip install pyLDAvis
!pip install nltk.tokenize

import pandas as pd
import nltk; nltk.download('stopwords')
import gensim
import re
import numpy as np
from pprint import pprint
import string
nltk.download('wordnet')
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import string

from sklearn.feature_extraction.text import CountVectorizer;
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
#import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import seaborn as sns
import random
import scipy.stats as stats
from scipy.stats import chi2_contingency
import sklearn.metrics as metrics
import requests 
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
#import colorama
import re
import scipy
from collections import Counter
from time import time
%matplotlib inline

import ipywidgets as widgets
import docx
from IPython.core.display import HTML, display, Javascript, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual

import warnings
warnings.filterwarnings("ignore");

ERROR: Could not find a version that satisfies the requirement nltk.tokenize (from versions: none)
ERROR: No matching distribution found for nltk.tokenize
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#! pip install voila
import voila

# Scraping BBC for all articles 

In [0]:
html = requests.get("https://www.bbc.com/").text # fstrings require Python 3.6+
soup = BeautifulSoup(html, "html.parser")
All_links = [element.get('href') for element in soup.findAll('a')]

In [18]:
All_links[0:5]

['https://www.bbc.co.uk',
 '#orb-modules',
 'https://www.bbc.co.uk/accessibility/',
 'https://account.bbc.com/account',
 '#']

In [0]:
df = pd.DataFrame(All_links,columns=['links'])

In [20]:
df.shape

(274, 1)

In [0]:
df_internal_links = df[df['links'].str.contains('bbc')]

In [22]:
df_internal_links.shape

(106, 1)

In [23]:
df_internal_links.head()

,links
0,https://www.bbc.co.uk
2,https://www.bbc.co.uk/accessibility/
3,https://account.bbc.com/account
5,https://www.bbc.co.uk
6,https://www.bbc.co.uk/news


In [24]:
range(0,len(df_internal_links))

range(0, 106)

In [0]:
content=[]

for i in range(0,len(df_internal_links)):
    url = df_internal_links.iloc[i,0]
    try:
        r = requests.get(url) 
        #print('Web site exists')
        #print(r.content) 
        page = r.text
        soup = BeautifulSoup(page)
        # Get headline
        #headline = soup.find('h1').get_text()
        # Get text from all <p> tags.
        p_tags = soup.find_all('p')
        # Get the text from each of the “p” tags and strip surrounding whitespace.
        p_tags_text = [tag.get_text().strip() for tag in p_tags]
        # Filter out sentences that contain newline characters '\n' or don't contain periods.
        sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
        sentence_list = [sentence for sentence in sentence_list if '.' in sentence]
        # Combine list items into string.
        article = ' '.join(sentence_list)
        content.append(article)
    except requests.exceptions.RequestException as e:  
        print("error")
        continue     

In [26]:
content[0]

"The MUAs work on an advertising campaign for sports fashion retailer JD. In rural Ireland, popular Connell and aloof Marianne begin an intense, secret affair. Eve tries to rebuild her life, having been shot and left for dead in Rome by Villanelle. Following Prince William's campaign to get us to open up about mental health issues."

In [0]:
df = pd.DataFrame(content,columns=['content'])

In [0]:
df = df.dropna()

In [29]:
df.head()

,content
0,The MUAs work on an advertising campaign for s...
1,"This site provides accessibility help, enablin..."
2,
3,The MUAs work on an advertising campaign for s...
4,The US city sees a third night of unrest spark...


# Data cleaning-removing punctuations,quotations,spaces

In [0]:
stop_words = stopwords.words('english')

In [0]:
Cleaned_Data = []
for i in range(0, len(df)):
    article = re.sub('[^a-zA-Z0-9]{3,}', ' ', str(df['content'][i]))
    article = re.sub('\[.*?\]', '', article)
    article = re.sub('[%s]' % re.escape(string.punctuation), '', article)
    article = re.sub('\w*\d\w*', '', article)
    article = re.sub('[‘’“”…]', '', article)
    article = re.sub('\n', '', article)
    article = article.lower()
    article = article.split()        
    lmtzr = WordNetLemmatizer()
    article =  [lmtzr.lemmatize(word) for word in article if word not in (stop_words)]
    article = ' '.join(article)
    Cleaned_Data.append(article)

In [0]:
final_words = []

In [0]:
for j in range(0,len(Cleaned_Data)):
    query = Cleaned_Data[j]
    querywords = query.split()
    query_yes = [word for word in querywords if len(word)> 4]
    final_words.append(' '.join(query_yes))

In [34]:
len(final_words)

106

In [35]:
final_words[4]

'third night unrest sparked death george floyd police custody third night unrest sparked death george floyd police custody jimenez reporting protest minneapolis detained officer first twitter hidden tweet president profile behind warning president follows decision twitter factcheck notice tweet stauffer behaviour became difficult critic accuse exploitation zookeeper caitlin henderson ended lockdown spider official people disease april initially announced norwegian holiday others country midjune swede several incident crackdown oromia amnesty international resident holding overseas passport could route citizenship several incident crackdown oromia amnesty international resident holding overseas passport could route citizenship cannabis residue found altar ancient desert temple israel school reopened forced spike actress gabrielle union series toxic environment email haveyoursaybbccouk'

# Running the model

Vectorized the data and transformed to input in the final LDA model.

In [0]:
cv = CountVectorizer(stop_words='english')

In [0]:
data_cv = cv.fit_transform(final_words)

In [0]:
lda = LatentDirichletAllocation(n_components=10, learning_method='online',verbose=False)
data_lda = lda.fit_transform(data_cv)

In [0]:
def selected_topics(model, cv, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(cv.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

**Topics as output with their probability scores**

In [40]:
print("LDA Model:")
selected_topics(lda, cv)

LDA Model:
Topic 0:
[('people', 20.55164540762188), ('farmer', 18.926143879380547), ('fruit', 15.708053292302825), ('world', 11.673960115326539), ('zealand', 11.105455828464821), ('harvest', 11.080435269239073), ('lockdown', 10.63963040269747), ('labour', 9.645665081953467), ('field', 9.62173651805778), ('automation', 8.121535925433411)]
Topic 1:
[('taster', 5.100074353907571), ('people', 2.3773207211451393), ('really', 1.9553794725757592), ('research', 1.953981606178365), ('development', 1.9523003980449858), ('future', 1.9277482830712274), ('explainers', 1.9189245315837455), ('regularly', 1.910947097209835), ('content', 1.9101558672548364), ('adult', 1.9072116416613998)]
Topic 2:
[('recipe', 12.36753983956357), ('world', 9.535706823700785), ('series', 7.725198548672479), ('ingredient', 7.513622293359035), ('podcast', 6.651752949105763), ('favourite', 5.915226689909292), ('watch', 5.341894684620046), ('solution', 5.2630751392359665), ('information', 5.135669863011264), ('vegan', 5.1336

# Predicting topic through a Web App

In [49]:
Upload = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

Upload

FileUpload(value={}, description='Upload')

In [0]:
with open("input.docx", "w+b") as i:
    i.write(Upload.data[0])

In [0]:
text = docx.Document('input.docx')

In [0]:
all_paras = text.paragraphs

In [53]:
text_data=[]
for para in all_paras:
    print(para.text)
    text_data.append(para.text)


To Hiring Manager,

I possess close to two and a half years of experience with Ameriprise Financial, a Fortune 500 financial services firm. My role there was to provide solutions to business leaders and external business partners leveraging analytical tools to derive actionable insights. I have led projects with large unstructured data in the advisors’ domain to realize the potential of coaching, certifications in financial planning and strategized transfers, adding revenue to the business. Having independently led projects in the past I have had the luxury to identify novel analytical directions and drive consumer focused, revenue generating solutions. Moreover, I have had hands on experience with SAS, Python, SQL, Excel, Tableau and R and I am also a SAS certified professional. 

     Post my work experience, I started a master’s in Business Analytics at the University of Cincinnati’s fast-paced environment. I worked considerably on challenging projects with a wide range of objectiv

In [0]:
article = re.sub('[^a-zA-Z0-9]{3,}', ' ', str(text_data))
article = re.sub('\[.*?\]', '', article)
article = re.sub('[%s]' % re.escape(string.punctuation), '', article)
article = re.sub('\w*\d\w*', '', article)
article = re.sub('[‘’“”…]', '', article)
article = re.sub('\n', '', article)
article = article.lower()
article = article.split()        
lmtzr = WordNetLemmatizer()
article =  [lmtzr.lemmatize(word) for word in article if word not in (stop_words)]
article = ' '.join(article)
Cleaned_Data = article

In [55]:
Cleaned_Data

'hiring manager posse close two half year experience ameriprise financial fortune financial service firm role provide solution business leader external business partner leveraging analytical tool derive actionable insight led project large unstructured data advisor domain realize potential coaching certification financial planning strategized transfer adding revenue business independently led project past luxury identify novel analytical direction drive consumer focused revenue generating solution moreover hand experience sa python sql excel tableau r also sa certified professional post work experience started master business analytics university cincinnati fastpaced environment worked considerably challenging project wide range objective like topic modelling predicting price rating business identifying customer likely churn simulation modelling creating interactive tableau dashboard process developed strong hold identifying key question project brushed concept like predictive analysis

In [56]:
type(Cleaned_Data)

str

In [0]:
text_list=[Cleaned_Data]

In [58]:
text_list

['hiring manager posse close two half year experience ameriprise financial fortune financial service firm role provide solution business leader external business partner leveraging analytical tool derive actionable insight led project large unstructured data advisor domain realize potential coaching certification financial planning strategized transfer adding revenue business independently led project past luxury identify novel analytical direction drive consumer focused revenue generating solution moreover hand experience sa python sql excel tableau r also sa certified professional post work experience started master business analytics university cincinnati fastpaced environment worked considerably challenging project wide range objective like topic modelling predicting price rating business identifying customer likely churn simulation modelling creating interactive tableau dashboard process developed strong hold identifying key question project brushed concept like predictive analysi

In [0]:
data_pred = cv.fit_transform(text_list)
lda = LatentDirichletAllocation(n_components=1, learning_method='online',verbose=False)
data_lda = lda.fit_transform(data_pred);

In [0]:
#lda.components_

In [61]:
for idx, topic in enumerate(lda.components_):
        #print("Topic %d:" % (idx))
        print([(cv.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-9:-1]]) 

[('business', 5.847246232590024), ('experience', 4.2223675733900246), ('project', 4.221565047394457), ('financial', 4.213626074489379), ('data', 3.4452555330862653), ('modelling', 3.4037473348447125), ('identifying', 2.6343787568976245), ('revenue', 2.6334688093709406)]


In [62]:
max(lda.components_[0])

5.847246232590024

In [63]:
print(sorted(lda.components_[0],reverse=True)[0:9])

[5.847246232590024, 4.2223675733900246, 4.221565047394457, 4.213626074489379, 3.4452555330862653, 3.4037473348447125, 2.6343787568976245, 2.6334688093709406, 2.625132040108786]


## function

In [0]:
def topic_model_func(doc):

    article = re.sub('[^a-zA-Z0-9]{3,}', ' ', str(text_data))
    article = re.sub('\[.*?\]', '', article)
    article = re.sub('[%s]' % re.escape(string.punctuation), '', article)
    article = re.sub('\w*\d\w*', '', article)
    article = re.sub('[‘’“”…]', '', article)
    article = re.sub('\n', '', article)
    article = article.lower()
    article = article.split()        
    lmtzr = WordNetLemmatizer()
    article =  [lmtzr.lemmatize(word) for word in article if word not in (stop_words)]
    article = ' '.join(article)
    Cleaned_Data = article

    text_list=[Cleaned_Data]

    data_pred = cv.fit_transform(text_list)
    lda = LatentDirichletAllocation(n_components=1, learning_method='online',verbose=False)
    data_lda = lda.fit_transform(data_pred)

    for idx, topic in enumerate(lda.components_):
           print([(cv.get_feature_names()[i]) for i in topic.argsort()[:-9:-1]])

In [67]:
topic_model_func(doc=text_data)

['business', 'financial', 'experience', 'project', 'modelling', 'data', 'like', 'led']


In [0]:
Button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

In [69]:
Button

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

['business', 'financial', 'experience', 'project', 'modelling', 'data', 'collaboration', 'professional']


In [0]:
Button.on_click(topic_model_func)